In [0]:
dbutils.notebook.exit(1)

In [0]:
%sql 
DESCRIBE gold_fnb_sales

In [0]:
# %sql
# delete 
# FROM silver_fnb_sales
# WHERE RETAILER = 'COSTCO'
#   AND CATEGORY = 'Beverages'
#   AND BRAND IN ('Red Bull','Monster Energy','C4 Energy','Tropicana','Paper Boat')

In [0]:
# %sql
# delete 
# FROM bronze_fnb_sales
# WHERE RETAILER = 'COSTCO'
#   AND CATEGORY = 'Beverages'
#   AND BRAND IN ('Red Bull','Monster Energy','C4 Energy','Tropicana','Paper Boat')

In [0]:
# %sql
# delete 
# FROM bronze_fnb_sales
# WHERE RETAILER = 'CVS'
#   AND CATEGORY = 'Beverages'
#   AND BRAND IN ('The Cold Pressed Juicery','Paper Boat','Amul','Horlicks','Monster Energy','C4 Energy')

In [0]:
# %sql
# delete 
# FROM silver_fnb_sales
# WHERE RETAILER = 'CVS'
#   AND CATEGORY = 'Dairy_Products'
#   AND BRAND IN ('Amul Cheese','Amul Milk','Amul Masti')

## # **TABLE VOLUME**

In [0]:
# Volume pillar extractor: table-level volume metrics + partition skew heuristic
import re, math, uuid
from datetime import datetime, timezone
from pyspark.sql import functions as F
from pyspark.sql import types as T

# CONFIG: tables to scan
DEFAULT_DB = "default"
tables = {
    "Bronze": f"{DEFAULT_DB}.bronze_fnb_sales",
    "Silver": f"{DEFAULT_DB}.silver_fnb_sales",
    "old":   f"{DEFAULT_DB}.gold_fnb_sales"
}
# toggle counts (row_count uses df.count() and may be slow on very large tables)
compute_counts = True

# Helpers
def describe_detail(full_table):
    try:
        rows = spark.sql(f"DESCRIBE DETAIL {full_table}").collect()
        if rows:
            d = rows[0].asDict()
            # Delta fields: 'location', 'sizeInBytes', 'partitionColumns'
            location = d.get("location") or d.get("Location")
            size = d.get("sizeInBytes") or d.get("sizeInBytes".lower()) or None
            partitions = d.get("partitionColumns") or d.get("partitioncolumns") or []
            return {"location": location, "sizeInBytes": size, "partitionColumns": partitions}
    except Exception:
        return {"location": None, "sizeInBytes": None, "partitionColumns": None}

def get_table_size_from_path(path):
    # sum sizes recursively under path when DESCRIBE DETAIL didn't give sizeInBytes
    try:
        def recurse_sum(p):
            try:
                files = dbutils.fs.ls(p)
            except Exception:
                return 0
            s = 0
            for f in files:
                if f.isDir():
                    s += recurse_sum(f.path)
                else:
                    mt = getattr(f, "size", None) or getattr(f, "length", None) or getattr(f, "modificationTime", None)
                    # Databricks FileInfo sometimes uses 'size' or 'length'; fallback if missing
                    if hasattr(f, "size"):
                        s += int(f.size)
                    elif hasattr(f, "length"):
                        s += int(f.length)
                    elif getattr(f, "modificationTime", None):
                        # can't derive size from mod time; skip
                        continue
            return s
        total = recurse_sum(path)
        return total if total>0 else None
    except Exception:
        return None

def partition_skew_heuristic(full_table, part_cols):
    # Use only first partition column for heuristic to keep it fast
    if not part_cols:
        return {"skew_label": "Unknown (no partitions)", "max_partition_count": None, "median_partition_count": None, "ratio": None}
    pcol = part_cols[0]
    try:
        # compute counts per partition value (careful with many partitions; limit to top N unique partitions to avoid explosion)
        counts_df = spark.sql(f"SELECT {pcol} AS p, COUNT(1) AS cnt FROM {full_table} GROUP BY {pcol}")
        # collect counts as list of ints
        counts = [row["cnt"] for row in counts_df.select("cnt").collect()]
        if not counts:
            return {"skew_label": "Unknown (no data)", "max_partition_count": 0, "median_partition_count": 0, "ratio": None}
        counts_sorted = sorted(counts)
        max_c = counts_sorted[-1]
        # median
        n = len(counts_sorted)
        if n % 2 == 1:
            median_c = counts_sorted[n//2]
        else:
            median_c = (counts_sorted[n//2 - 1] + counts_sorted[n//2]) / 2.0
        # avoid zero division
        median_safe = median_c if median_c>0 else 1
        ratio = float(max_c) / float(median_safe)
        if ratio > 1.5:
            label = "Skewed"
        elif ratio > 3:
            label = "Partially skewed"
        else:
            label = "Normal"
        return {"skew_label": label, "max_partition_count": int(max_c), "median_partition_count": float(median_c), "ratio": round(ratio,2)}
    except Exception:
        return {"skew_label": "Unknown (skew computation failed)", "max_partition_count": None, "median_partition_count": None, "ratio": None}

# Main loop: collect metrics
out = []
for layer, full_table in tables.items():
    rec = {"layer": layer, "full_table": full_table}
    # describe detail
    det = describe_detail(full_table)
    rec["storage_path"] = det.get("location")
    rec["table_size_bytes_describe"] = det.get("sizeInBytes")

    # row/col counts
    if compute_counts:
        try:
            df = spark.table(full_table)
            rec["row_count"] = int(df.count())
            rec["col_count"] = len(df.columns)
        except Exception as e:
            rec["row_count"] = None
            rec["col_count"] = None
            rec["notes"] = ("count_failed:" + str(e)) if rec.get("notes") is None else rec["notes"] + ";count_failed:" + str(e)
    else:
        rec["row_count"] = None
        rec["col_count"] = None

   

    # partition skew
    part_cols = det.get("partitionColumns") or []
    if isinstance(part_cols, str):
        # sometimes comes as string representation
        try:
            part_cols = eval(part_cols)
        except Exception:
            part_cols = [part_cols]
    skew = partition_skew_heuristic(full_table, part_cols)
    rec.update(skew)

    out.append(rec)

# Convert to DataFrame with explicit schema
schema = T.StructType([
    T.StructField("layer", T.StringType(), True),
    T.StructField("full_table", T.StringType(), True),
    T.StructField("storage_path", T.StringType(), True),
    T.StructField("table_size_bytes_describe", T.LongType(), True),
    
    T.StructField("row_count", T.LongType(), True),
    T.StructField("col_count", T.IntegerType(), True),
    T.StructField("skew_label", T.StringType(), True),
    T.StructField("max_partition_count", T.LongType(), True),
    T.StructField("median_partition_count", T.DoubleType(), True),
    T.StructField("ratio", T.DoubleType(), True),
    
])
rows = []
for r in out:
    rows.append((
        r.get("layer"),
        r.get("full_table"),
        r.get("storage_path"),
        r.get("table_size_bytes_describe"),
      
        r.get("row_count"),
        r.get("col_count"),
        r.get("skew_label"),
        r.get("max_partition_count"),
        r.get("median_partition_count"),
        r.get("ratio"),
        
    ))
volume_metrics_df = spark.createDataFrame(rows, schema=schema)

display(volume_metrics_df)
# volume_metrics_df is the PySpark DataFrame you can write to Delta like:
# volume_metrics_df.write.format("delta").mode("overwrite").saveAsTable("do_tool.volume_metrics")



In [0]:
# Save 'volume_metrics' DataFrame as Delta table 'workspace.default.table_volume'
from datetime import datetime
import uuid, os

df = volume_metrics_df
CAT, SCH, TBL = "workspace", "default", "table_volume"
FULL = f"{CAT}.{SCH}.{TBL}"

try:
    # df.write.format("delta").mode("overwrite").saveAsTable(FULL)
    print(f"✅ Successfully wrote table: {FULL}")
except Exception as e:
    print(f"⚠️ Write to {FULL} failed: {str(e).splitlines()[0]}")
    # Fallback: save in user folder (always writable)
    try:
        user = spark.sql("SELECT current_user() as u").collect()[0]["u"]
    except Exception:
        user = os.environ.get("USER") or "unknown_user"
    safe_user = user.replace("@", "_at_").replace(" ", "_")
    path = f"/Users/{safe_user}/do_tool/{TBL}_{uuid.uuid4().hex}"
    # df.write.format("delta").mode("overwrite").save(path)
    print(f"✅ Saved Delta files to: {path}")
    print(f"\nIf you want it registered as a shared table, ask an admin to run:\n"
          f"CREATE TABLE {FULL} USING DELTA LOCATION '{path}';")


## **DATA VOLUME**

In [0]:
# Data Volume Pillar: compare current vs previous Delta table row counts by (RETAILER, CATEGORY)
from pyspark.sql import functions as F
from pyspark.sql import types as T

# CONFIG: your tables and how many versions to go back
DEFAULT_DB = "default"
layers = {
    "BRONZE": f"{DEFAULT_DB}.bronze_fnb_sales",
    "SILVER": f"{DEFAULT_DB}.silver_fnb_sales",
    "GOLD":   f"{DEFAULT_DB}.gold_fnb_sales"
}
previous_version_offset = 1   # Compare with version N-1

def get_version_number(table_name):
    try:
        df_hist = spark.sql(f"DESCRIBE HISTORY {table_name}")
        versions = df_hist.select("version").orderBy(F.desc("version")).collect()
        if len(versions) < 2:
            return None, None  # no previous version
        curr = int(versions[0]["version"])
        prev = 0
        return curr, prev
    except Exception as e:
        print(f"⚠️ Could not read history for {table_name}: {e}")
        return None, None

def get_row_counts(table_name, version):
    """Return (RETAILER, CATEGORY, row_count) at given version"""
    try:
        df = spark.read.format("delta").option("versionAsOf", version).table(table_name)
        agg_df = (
            df.filter(F.col("RETAILER").isNotNull() & F.col("CATEGORY").isNotNull())
              .groupBy("RETAILER", "CATEGORY")
              .agg(F.count("*").alias("row_count"))
        )
        return agg_df
    except Exception as e:
        print(f"⚠️ Could not load version {version} for {table_name}: {e}")
        return None

# collect results
results = []

for layer, full_table in layers.items():
    curr_v, prev_v = get_version_number(full_table)
    if curr_v is None or prev_v is None:
        print(f"Skipping {layer} — not enough history yet.")
        continue

    curr_df = get_row_counts(full_table, curr_v)
    prev_df = get_row_counts(full_table, prev_v)
    if curr_df is None or prev_df is None:
        continue

    joined = (
        curr_df.alias("curr")
        .join(prev_df.alias("prev"),
              on=["RETAILER", "CATEGORY"],
              how="outer")
        .select(
            F.coalesce(F.col("curr.RETAILER"), F.col("prev.RETAILER")).alias("RETAILER"),
            F.coalesce(F.col("curr.CATEGORY"), F.col("prev.CATEGORY")).alias("CATEGORY"),
            F.col("curr.row_count").alias("row_count"),
            F.col("prev.row_count").alias("row_count_prev"),
        )
        .withColumn("layer", F.lit(layer))
    )

    joined = joined.withColumn(
        "difference_percent",
        F.when(F.col("row_count_prev").isNotNull() & (F.col("row_count_prev") != 0),
               ((F.col("row_count") - F.col("row_count_prev")) / F.col("row_count_prev") * 100)
        ).otherwise(None)
    )

    joined = joined.withColumn(
        "growth_status",
        F.when(F.col("difference_percent") > 0, F.lit("Growth"))
         .when(F.col("difference_percent") < 0, F.lit("Drop"))
         .otherwise(F.lit("No Change"))
    )

    results.append(joined)

# combine all layers
if results:
    data_volume_df = results[0]
    for df in results[1:]:
        data_volume_df = data_volume_df.unionByName(df, allowMissingColumns=True)
else:
    data_volume_df = spark.createDataFrame([], T.StructType([
        T.StructField("layer", T.StringType()),
        T.StructField("RETAILER", T.StringType()),
        T.StructField("CATEGORY", T.StringType()),
        T.StructField("row_count", T.LongType()),
        T.StructField("row_count_prev", T.LongType()),
        T.StructField("difference_percent", T.DoubleType()),
        T.StructField("growth_status", T.StringType())
        
    ]))

display(data_volume_df)


In [0]:
# Save 'data_volume' DataFrame as Delta table 'workspace.default.data_volume'
from datetime import datetime
import uuid, os

df = data_volume_df
CAT, SCH, TBL = "workspace", "default", "data_volume"
FULL = f"{CAT}.{SCH}.{TBL}"

try:
    # df.write.format("delta").mode("overwrite").saveAsTable(FULL)
    print(f"✅ Successfully wrote table: {FULL}")
except Exception as e:
    print(f"⚠️ Write to {FULL} failed: {str(e).splitlines()[0]}")
    # Fallback: save in your user folder (always writable)
    try:
        user = spark.sql("SELECT current_user() as u").collect()[0]["u"]
    except Exception:
        user = os.environ.get("USER") or "unknown_user"
    safe_user = user.replace("@", "_at_").replace(" ", "_")
    path = f"/Users/{safe_user}/do_tool/{TBL}_{uuid.uuid4().hex}"
    # df.write.format("delta").mode("overwrite").save(path)
    print(f"✅ Saved Delta files to: {path}")
    print(f"\nIf you want it registered as a shared table, ask an admin to run:\n"
          f"CREATE TABLE {FULL} USING DELTA LOCATION '{path}';")
